In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import read_csv, concat
from tqdm import tqdm
sns.set_style('white')
sns.set_context('notebook', font_scale=1.33)
%matplotlib inline

In [5]:
## Load reject data.
reject = read_csv(os.path.join('..','data','reject.csv'))

## 
metrics = reject[['platform','subject','infreq']]

## Section 1: Construct Behavioral Metrics

In [2]:
## Load behavioral data.
data = read_csv(os.path.join('..','data','data.csv'))

## Restrict participants.
data = data.loc[data.subject.isin(reject.subject)]

#### 1.1 Accuracy

In [7]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute metrics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge with reject.
metrics = metrics.merge(reject[['subject','platform','accuracy']])


#### 1.2 WSLS

In [8]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Compute metrics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Merge with reject.
metrics = metrics.merge(reject[['subject','platform','wsls']])


In [9]:
data

,platform,subject,trial,correct,choice,key,rt,accuracy,outcome
0,mturk,02rwq76tp9lcye8vrl41llnh,1,2,1.0,38.0,3.809,0.0,0.0
1,mturk,02rwq76tp9lcye8vrl41llnh,2,2,2.0,39.0,0.210,1.0,1.0
2,mturk,02rwq76tp9lcye8vrl41llnh,3,2,2.0,39.0,0.494,1.0,1.0
3,mturk,02rwq76tp9lcye8vrl41llnh,4,2,2.0,39.0,2.104,1.0,1.0
4,mturk,02rwq76tp9lcye8vrl41llnh,5,2,0.0,37.0,1.329,0.0,1.0
...,...,...,...,...,...,...,...,...,...
36535,prolific,zs4sf8n0ws77fp0e0a75jflw,86,2,2.0,39.0,0.326,1.0,0.0
36536,prolific,zs4sf8n0ws77fp0e0a75jflw,87,2,2.0,39.0,0.224,1.0,1.0
36537,prolific,zs4sf8n0ws77fp0e0a75jflw,88,2,2.0,39.0,0.299,1.0,0.0
36538,prolific,zs4sf8n0ws77fp0e0a75jflw,89,2,2.0,39.0,0.249,1.0,1.0
